In [ ]:
import numpy as np
import matplotlib.pyplot as plt # version > 3.0
import pandas as pd
from datetime import datetime
from google.colab import drive
import math
plt.style.use('seaborn')

In [ ]:
path_to_sepsis3_single_stay_csv = "/content/Sepsis_Labs_Vitals_Demographic.csv"
sepsis_original = pd.read_csv(path_to_sepsis3_single_stay_csv)
path_to_sepsis3_many_stays_csv = "/content/sepsis_labs_avg.csv"
path_to_vitals_stays_csv = "/content/vitals_avg.csv"
sepsis_many_stays_df = pd.read_csv(path_to_sepsis3_many_stays_csv)
vitals_df = pd.read_csv(path_to_vitals_stays_csv)
print(sepsis_original.shape)
print(sepsis_many_stays_df.shape)
print(vitals_df.shape)
sepsis_original.info()
sepsis_original.describe()
sepsis_many_stays_df.describe()
sepsis_many_stays_df.info()

vitals_list = []

for column, _ in vitals_df.iteritems():
  vitals_list.append(column)

print(vitals_list)

sepsis_lab_list = []

for column, _ in sepsis_many_stays_df.iteritems():
  sepsis_lab_list.append(column)

print(sepsis_lab_list)

def find_first_non_null_sepsis_lab(subject_id, stay_id, column_name):
  modified_sepsis_lab_1 = sepsis_many_stays_df[sepsis_many_stays_df["subject_id"] == subject_id]
  modified_sepsis_lab_2 = modified_sepsis_lab_1[modified_sepsis_lab_1["stay_id"] == stay_id]
  for _, row in modified_sepsis_lab_2.iterrows():
    if type(row[column_name+"_avg"]) != str and not math.isnan(row[column_name+"_avg"]):
      return row[column_name+"_avg"]
    else:
      return None

def find_first_non_null_vital(subject_id, stay_id, column_name):
  modified_vitals_1 = vitals_df[vitals_df["subject_id"] == subject_id]
  modified_vitals_2 = modified_vitals_1[modified_vitals_1["stay_id"] == stay_id]
  for _, row in modified_vitals_2.iterrows():
    if type(row[column_name+"_avg"]) != str and not math.isnan(row[column_name+"_avg"]):
      return row[column_name+"_avg"]
    else:
      return None


sepsis_non_null_df = sepsis_original.copy(deep=True)

for column,_ in sepsis_original.iteritems():
  for i,row in sepsis_original.iterrows():
    if column+"_avg" in sepsis_lab_list and math.isnan(row[column]):
      non_null_value = find_first_non_null_sepsis_lab(row["subject_id"], row["stay_id"], column)
      if non_null_value is not None:
        print("Supposed to be non null: ",non_null_value)
      sepsis_non_null_df.at[i, column] = non_null_value
    elif column+"_avg" in vitals_list and math.isnan(row[column]):
      non_null_value = find_first_non_null_vital(row["subject_id"], row["stay_id"], column)
      if non_null_value is not None:
        print("Supposed to be non null: ",non_null_value)
      sepsis_non_null_df.at[i, column] = non_null_value
    else:
      continue

sepsis_non_null_df.info()
sepsis_non_null_df.to_csv("/content/Modified_All.csv")
  


Streaming output truncated to the last 5000 lines.
Supposed to be non null:  101.83333333333331
Supposed to be non null:  97.35294117647058
Supposed to be non null:  178.5769230769231
Supposed to be non null:  152.22222222222226
Supposed to be non null:  82.0
Supposed to be non null:  128.07999999999998
Supposed to be non null:  139.39473684210526
Supposed to be non null:  105.0
Supposed to be non null:  131.0
Supposed to be non null:  134.5
Supposed to be non null:  116.5
Supposed to be non null:  107.5
Supposed to be non null:  123.8095238095238
Supposed to be non null:  101.0
Supposed to be non null:  147.2
Supposed to be non null:  170.83333333333334
Supposed to be non null:  163.2
Supposed to be non null:  110.0
Supposed to be non null:  90.33333333333331
Supposed to be non null:  112.8
Supposed to be non null:  101.0
Supposed to be non null:  163.06666666666666
Supposed to be non null:  128.1
Supposed to be non null:  122.4
Supposed to be non null:  118.07142857142857
Supposed to

In [ ]:
path_to_csv = "/content/Sepsis_Lab_and_Vital_with_Stay_Avg.csv"
updated_df = pd.read_csv(path_to_csv)
updated_df_copy = updated_df.copy(deep=True)
updated_df_copy_1 = updated_df_copy.fillna(value={"pao2fio2ratio_novent": 450, "pao2fio2ratio_vent": 450, "rate_dobutamine":0 , "rate_epinephrine":0, "rate_norepinephrine":0, "rate_dopamine":0, "meanbp_min":100, "gcs_min":16, "uo_24hr": 550, 
                                            "bilirubin_max":0.9, "creatinine_max": 0.9, "platelet_min":180, "heart_rate":80, "sbp":120, "mbp":100, "dbp":80, "sbp_ni":120, "mbp_ni":100, "dbp_ni":80, 
                                            "temperature":97, "glucose":140, "spo2":95, "resp_rate":14 })

updated_df_copy_1.describe()
updated_df_copy_1.to_csv("/content/No_NaN_Values_New.csv")


In [ ]:
# sepsis_df["death_time"]=sepsis_df["death_time"].astype(str)
# sepsis_df["sofa_time"]=sepsis_df["sofa_time"].astype(str)

In [ ]:
format = '%Y-%m-%d %H:%M:%S'
for i in range(sepsis_df.shape[0]):
  if sepsis_df["death_flag"][i] == 1 and type(sepsis_df["death_time"][i]) is str:
    time_diff = datetime.strptime(sepsis_df["death_time"][i],format) - datetime.strptime(sepsis_df["sofa_time"][i],format)
    time_diff_in_days = time_diff.total_seconds()/(360*24)
    sepsis_df["time_to_death"][i]= time_diff_in_days
  else:
    sepsis_df["time_to_death"][i]=  None

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Getting all the information for a particular subject ID

In [ ]:
sepsis_df[sepsis_df['subject_id'] == 10017531]

,subject_id,stay_id,sofa_time,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_dobutamine,rate_epinephrine,rate_norepinephrine,rate_dopamine,meanbp_min,gcs_min,uo_24hr,bilirubin_max,creatinine_max,platelet_min,death_time,death_flag,heart_rate,sbp,mbp,dbp,sbp_ni,mbp_ni,dbp_ni,temperature,glucose,spo2,resp_rate,subject_id_1,hadm_id,stay_id_1,los_hospital,admission_age,hospstay_seq,first_hosp_stay,time_to_death,ICULOS,icustay_seq,first_icu_stay
0,10017531,32136798,2159-12-26 21:00:00,NaN,NaN,NaN,NaN,0.100081,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,0,123.0,107.0,73.0,59.0,NaN,NaN,NaN,NaN,NaN,98.0,28.0,10017531,21095812,32136798,40,64,2,False,None,39.83,1,True
1,10017531,35526828,2159-09-23 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,108.0,14.0,NaN,NaN,NaN,NaN,NaN,0,115.0,155.0,108.0,91.0,155.0,108.0,91.0,NaN,NaN,97.0,27.0,10017531,22580355,35526828,32,64,1,True,None,31.79,1,True


In [ ]:
for i in sepsis_df['subject_id'].unique():
  df = sepsis_df[sepsis_df['subject_id'] == i]
  for j in df.shape[0]:
    print(j)

Streaming output truncated to the last 5000 lines.
    subject_id   stay_id  ... icustay_seq  first_icu_stay
48    11713021  31148453  ...           1            True
49    11713021  33267926  ...           1            True

[2 rows x 39 columns]
    subject_id   stay_id  ... icustay_seq  first_icu_stay
50    11740539  38596870  ...           1            True

[1 rows x 39 columns]
    subject_id   stay_id  ... icustay_seq  first_icu_stay
51    11747893  32792276  ...           2           False
52    11747893  32344065  ...           1            True

[2 rows x 39 columns]
    subject_id   stay_id  ... icustay_seq  first_icu_stay
53    11767995  36269947  ...           1            True

[1 rows x 39 columns]
    subject_id   stay_id  ... icustay_seq  first_icu_stay
54    11770234  38001330  ...           1            True

[1 rows x 39 columns]
    subject_id   stay_id  ... icustay_seq  first_icu_stay
55    11803776  38717988  ...           2           False

[1 rows x 39 columns]

KeyboardInterrupt: ignored

In [ ]:
# df = sepsis_df[sepsis_df['subject_id'] == 11713021]
# for j in range(df.shape[0]):
#     print(df[0])

In [ ]:
# for index, row in df.iterrows():
    # print(row)

In [ ]:
print(df['gcs_min'].argmax())
print(df['gcs_min'].argmax())

1